In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [2]:
url = "https://raw.githubusercontent.com/maxxxxc/SIR-Summer-2023/main/dataset/Epileptic%20Seizure%20Recognition.csv"
df = pd.read_csv(url)

X = df.drop("y", axis=1)
y = df["y"]

X = X.drop("Unnamed", axis=1)

y = y.replace({4 : 0, 3 : 0, 2 : 0, 5 : 0})
y.value_counts()

0    9200
1    2300
Name: y, dtype: int64

In [3]:
#divide

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
#param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
#svm = SVC(kernel='rbf')

#grid_search = GridSearchCV(svm, param_grid, cv=5)
#grid_search.fit(X_train_scaled, y_train)

#best_params = grid_search.best_params_
#best_C = best_params['C']
#best_gamma = best_params['gamma']

best_C = 1
best_gamma = 1

#Fit the SVM with the best parameters:
svm = SVC(kernel='rbf', C=best_C, gamma=best_gamma)

In [7]:
svm.fit(X_train_scaled, y_train)

SVC(C=1, gamma=1)

In [8]:
# Make predictions on the test set
y_test_pred = svm.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", accuracy)

# Prediction rule is essentially: 1 if distance > 0
distances = svm.decision_function(X_test_scaled)
tmp = np.where(distances > 0, 1, 0)
print("Match between Prediction and Rule based on Distance")
accuracy_score(tmp, y_test_pred)

Test Accuracy: 0.7939130434782609
Match between Prediction and Rule based on Distance


1.0

In [9]:
from sklearn.calibration import CalibratedClassifierCV

In [10]:
svm_platt = CalibratedClassifierCV(svm)
svm_platt.fit(X_train_scaled, y_train)
y_test_prob = svm_platt.predict_proba(X_test_scaled)

In [11]:
y_test_pred = np.where(y_test_prob[:,1] > 0.5, 1, 0)
accuracy_score(y_test, y_test_pred)

0.7939130434782609

In [12]:
def svm_iterate_process(X, y):
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    
    # Define the number of batches
    num_batches = 11
    
    # Randomly shuffle the data indices
    indices = np.random.permutation(len(X_train))
    
    # Calculate the batch size
    batch_size = len(X_train) // num_batches
    
    # Make predictions on the test set using majority voting
    preds_voting = np.zeros(len(y_test))
    # Make predictions on the test set using average of logit
    preds_distance = np.zeros(len(y_test))
    #Make predictions on the test set using average of probs
    preds_prob = np.zeros(len(y_test))

    
    # Split the training data into batches, fit a logistic regression model on each batch
    for i in range(num_batches):
        # Calculate the starting and ending indices for the current batch
        start_index = i * batch_size
        end_index = (i + 1) * batch_size
        
        # Select the current batch for training
        X_batch = X_train.iloc[indices[start_index:end_index]]
        y_batch = y_train.iloc[indices[start_index:end_index]]
        
        scaler = StandardScaler()
        X_batch_scaled = scaler.fit_transform(X_batch)
        X_test_scaled = scaler.transform(X_test)
        
        
        # Create a support vector machine model
        param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
        svm = SVC(kernel='rbf')

        grid_search = GridSearchCV(svm, param_grid, cv=5)
        grid_search.fit(X_batch_scaled, y_batch)

        best_params = grid_search.best_params_
        best_C = best_params['C']
        best_gamma = best_params['gamma']
        svm = SVC(kernel='rbf', C=best_C, gamma=best_gamma)
        
        # Fit the model on the current batch
        svm.fit(X_batch_scaled, y_batch)
        
               
        # Accumulate the predictions using majority voting
        y_pred = svm.predict(X_test_scaled)
        preds_voting += (y_pred == 1)

    
        # Accumulate the predictions using majority voting
        y_pred = svm.decision_function(X_test_scaled)
        preds_distance += y_pred
        
        
        #Accumulate the probs
        svm_platt = CalibratedClassifierCV(svm)
        svm_platt.fit(X_batch_scaled, y_batch)
        y_pred = svm_platt.predict_proba(X_test_scaled)
        preds_prob += y_pred[:,1]
   
    
    accuracy = np.zeros(4)
    
    # Majority voting (selecting the most frequent prediction for each sample)
    final_predictions = np.where(preds_voting > num_batches / 2, 1, 0)
    accuracy[0] = accuracy_score(y_test, final_predictions)
    # Average of logit
    final_predictions = np.where(preds_distance > 0, 1, 0)
    accuracy[1] = accuracy_score(y_test, final_predictions)
    #Average of probs
    final_predictions = np.where(preds_prob / num_batches > 0.5, 1, 0)
    accuracy[2] = accuracy_score(y_test, final_predictions)
    
    # Train a model on all 11 batches of training data
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
    svm = SVC(kernel='rbf')

    grid_search = GridSearchCV(svm, param_grid, cv=5)
    grid_search.fit(X_train_scaled, y_train)

    best_params = grid_search.best_params_
    best_C = best_params['C']
    best_gamma = best_params['gamma']
    svm = SVC(kernel='rbf', C=best_C, gamma=best_gamma)
    
    svm.fit(X_train_scaled, y_train)
    y_pred = svm.predict(X_test_scaled)
    accuracy[3] = accuracy_score(y_test, y_pred)
    
    return accuracy

In [13]:
# Number of times to repeat the process
num_repeats = 50

# Initialize an empty matrix (10-by-4) to store accuracies
accuracies = np.zeros((num_repeats, 4))

seed = 42
# Repeat the process and store accuracies
for i in range(num_repeats):
    np.random.seed(seed)
    accuracies[i] = svm_iterate_process(X, y)
    seed += 2
    

# Print the accuracies
print("Accuracies:", accuracies)

Accuracies: [[0.79391304 0.79391304 0.93108696 0.93673913]
 [0.7976087  0.7976087  0.93369565 0.9376087 ]
 [0.80847826 0.80847826 0.93043478 0.935     ]
 [0.96413043 0.80456522 0.93195652 0.9373913 ]
 [0.80130435 0.80130435 0.93347826 0.93847826]
 [0.78891304 0.78891304 0.93826087 0.94108696]
 [0.80043478 0.80043478 0.9326087  0.93782609]
 [0.79413043 0.79413043 0.93086957 0.93652174]
 [0.9626087  0.79826087 0.9326087  0.93978261]
 [0.8023913  0.8023913  0.93108696 0.93543478]
 [0.7976087  0.7976087  0.9373913  0.93847826]
 [0.79630435 0.79630435 0.93630435 0.94065217]
 [0.79282609 0.79282609 0.92891304 0.9326087 ]
 [0.80152174 0.80152174 0.9273913  0.93086957]
 [0.80717391 0.80717391 0.93391304 0.93826087]
 [0.80369565 0.80369565 0.93717391 0.94      ]
 [0.80065217 0.80065217 0.93695652 0.94130435]
 [0.79782609 0.79782609 0.93369565 0.93456522]
 [0.79782609 0.79782609 0.93130435 0.93434783]
 [0.79782609 0.79782609 0.9276087  0.93108696]
 [0.795      0.795      0.93282609 0.93652174]
 

In [18]:
np.mean(accuracies, axis = 0) 

array([0.80547826, 0.799     , 0.93292174, 0.93681739])

In [19]:
np.std(accuracies, axis = 0)

array([0.03248248, 0.00412219, 0.00299343, 0.00285926])

In [14]:
y_test_prob[:5,:]

array([[1.00000000e+00, 6.97091584e-13],
       [7.12112290e-01, 2.87887710e-01],
       [7.12109024e-01, 2.87890976e-01],
       [8.27893906e-01, 1.72106094e-01],
       [8.58826486e-01, 1.41173514e-01]])

In [15]:
from sklearn.linear_model import LogisticRegression
train_distances = svm.decision_function(X_train_scaled)
model = LogisticRegression(max_iter=500)
model.fit(train_distances.reshape(-1, 1), y_train)
tmp = model.predict_proba(distances.reshape(-1, 1))

In [16]:
tmp[:5,:]

array([[0.98805953, 0.01194047],
       [0.98467621, 0.01532379],
       [0.98467621, 0.01532379],
       [0.98475861, 0.01524139],
       [0.98478417, 0.01521583]])

In [17]:
tmp1 = np.where(tmp[:,1] > 0.5, 1, 0)
accuracy_score(y_test, tmp1)

0.7939130434782609